# Introduction to TensorFlow: Model saving and restoring

#### Note: 
The way serialization is carried out in TF is via *checkpoint files* (usually .ckpt). The way this is done has changed quite a lot from version to version. Most chnages were mainly adding to the previous functionality and increase scope. This was true till version 0.12(the latest) when they changed their way of doing this 
more drastically and unfortunately not compatible with previous of Tensorflow.

Prior to r0.12, the saver would produce *two* files:
```
name_of_ckpt.ckpt
name_of_ckpt.ckpt.meta
```
And a checkpoint file << checkpoint >> (one per folder of ckpt files). You would explicilty save and restore << name_of_ckpt.ckpt >>. See screenshot below (for the models saved in this tutorial)

![Image](screenshot_ckpts.png)

From r0.12, we would get, by default, *three* similar files:
```
name_of_ckpt.ckpt.data00000-of-00001
name_of_ckpt.ckpt.meta
name_of_ckpt.ckpt.index
```    

#### Revert to old version
If you are using r0.12, use: 
```
saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)
```
to revert to the previous way of serializing.


#### More info:
For more information and (more advanced) functionality, check out saving and restoring documentation:
[r0.12](https://www.tensorflow.org/api_docs/python/state_ops/saving_and_restoring_variables)
[<r0.12](https://www.tensorflow.org/versions/r0.11/how_tos/variables/#saving_and_restoring)

In [1]:
# for compatibility 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
# import tensorflow
import tensorflow as tf

In [3]:
# let create some variables
A = tf.Variable(tf.random_normal([ 3, 3]), name="A")
b = tf.Variable(tf.random_normal([ 3, 1]), name="b")
prod = tf.matmul(A,b)

# update value of b
norm = tf.sqrt(tf.reduce_sum(tf.square(prod), 0))
update_op = b.assign(prod/norm)

In [4]:
# init operation
init_op = tf.initialize_all_variables()

## Saving a model
Now, we create a Saver with *tf.train.Saver()* to manage all variables in the model (default).

In [5]:
saver = tf.train.Saver()

We launch a session, where will update the value of vector b and will save the session current state at the end.

In [6]:
name_of_checkpoint = "model.ckpt"

In [7]:
with tf.Session() as sess:
    sess.run(init_op)
    
    print("Initial values:")
    print(sess.run(A))
    print(sess.run(b))
    
    print("\n Starting updating")
    for _ in range(20):
        sess.run(update_op)
        print(sess.run(b).T)
        
    print("\n Saving model to: " + name_of_checkpoint)
    saver.save(sess,name_of_checkpoint)

Initial values:
[[ 0.83655119 -0.09394854  0.70128042]
 [-0.47292292 -0.02102241 -0.39281079]
 [ 0.15759951 -1.44999313  0.16121884]]
[[ 0.17820637]
 [ 0.39067861]
 [ 1.60208094]]

 Starting updating
[[ 0.84743553 -0.49493656 -0.19206959]]
[[ 0.57700801 -0.29273942  0.76247329]]
[[ 0.77455395 -0.41973001  0.47317335]]
[[ 0.72337323 -0.38563386  0.57272828]]
[[ 0.74005705 -0.3966085   0.5431549 ]]
[[ 0.73506904 -0.39331514  0.55224699]]
[[ 0.7365979  -0.39432344  0.54948384]]
[[ 0.73613292 -0.39401671  0.55032647]]
[[ 0.73627466 -0.3941102   0.55006987]]
[[ 0.73623151 -0.39408165  0.55014801]]
[[ 0.73624468 -0.39409041  0.55012417]]
[[ 0.73624057 -0.39408773  0.5501315 ]]
[[ 0.73624188 -0.39408854  0.55012923]]
[[ 0.73624146 -0.3940883   0.55012989]]
[[ 0.73624158 -0.39408839  0.55012971]]
[[ 0.73624152 -0.39408836  0.55012977]]
[[ 0.73624152 -0.39408836  0.55012977]]
[[ 0.73624152 -0.39408836  0.55012977]]
[[ 0.73624152 -0.39408836  0.55012977]]
[[ 0.73624152 -0.39408836  0.55012977]]


## Restoring a model
Can create a similar Saver, or even re-use the one above. I will create a new one.

In [8]:
saver2restore = tf.train.Saver()

Launch a session to restore and check the values. These should match the ones at the end of the session above

In [9]:
with tf.Session() as sess:
    sess.run(init_op)
    saver2restore.restore(sess, name_of_checkpoint)
    
    print("Restored values:")
    print(sess.run(A))
    print(sess.run(b))

Restored values:
[[ 0.83655119 -0.09394854  0.70128042]
 [-0.47292292 -0.02102241 -0.39281079]
 [ 0.15759951 -1.44999313  0.16121884]]
[[ 0.73624152]
 [-0.39408836]
 [ 0.55012977]]


## Saving only certain variables/parameters rather then the whole session
This is useful, when computational graph is quite complex/large but we are interested only in some of the values -- like results of some computations. In this case, we specify when creating the Saver which variable(s) we are interested in. 

For instance in the example above, we might only be interested in the value of the computed eigenvector b.

In [10]:
saver = tf.train.Saver({"eigenv": b})
name_of_checkpoint = 'b.ckpt'

In [11]:
with tf.Session() as sess:
    sess.run(init_op)
    
    print("Initial values:")
    print(sess.run(A))
    print(sess.run(b))
    
    print("\n Starting updating")
    for _ in range(20):
        sess.run(update_op)
        print(sess.run(b).T)
        
    print("\n Saving model to: " + name_of_checkpoint)
    saver.save(sess, name_of_checkpoint)

Initial values:
[[ 0.20424592 -1.70958638  0.30089229]
 [ 0.29054552 -0.92459875 -0.09486626]
 [-0.74833429  0.79828358 -1.8937732 ]]
[[ 0.21762475]
 [ 1.08739328]
 [ 0.06087504]]

 Starting updating
[[-0.8492986  -0.44821203  0.27892274]]
[[ 0.92037457  0.19203478 -0.3406367 ]]
[[-0.82844472  0.41683042  0.37407458]]
[[-0.73710787 -0.63393873  0.23408031]]
[[ 0.88439906  0.30820918 -0.35049304]]
[[-0.87657046  0.01016468  0.4811663 ]]
[[-0.12921663 -0.77510887 -0.61847341]]
[[ 0.74949956  0.49699917  0.43731239]]
[[-0.48024252 -0.24075778 -0.84344691]]
[[ 0.03368387  0.09198278  0.99519074]]
[[ 0.08056752 -0.09169988 -0.99252206]]
[[-0.07116823  0.11482391  0.99083328]]
[[ 0.04994953 -0.12636039 -0.99072599]]
[[-0.0409787   0.12846883  0.99086654]]
[[ 0.03987652 -0.12773077 -0.99100691]]
[[-0.04070875  0.12705277  0.99106026]]
[[ 0.04127938 -0.12683606 -0.99106443]]
[[-0.04142991  0.12683378  0.99105841]]
[[ 0.04141637 -0.12686552 -0.99105489]]
[[-0.04138702  0.12688127  0.99105412]]


## Restoring b
Please note that b gets restored correctly, while A - since it was not saved - will be re-intialize to a random value.

In [12]:
with tf.Session() as sess:
    sess.run(init_op)
    saver.restore(sess, name_of_checkpoint)
    
    print("Restored values:")
    print(sess.run(A))
    print(sess.run(b))

Restored values:
[[-0.33163938 -0.0972058   0.86966449]
 [ 0.90413129 -0.31787494  0.85678315]
 [ 0.63517827 -0.8100847   0.22602408]]
[[-0.04138702]
 [ 0.12688127]
 [ 0.99105412]]
